# Web Scraping Homework - Mission to Mars

## Step 1 - Scraping

In [1]:
#####################################################################################
#                                                                                   #
# Import all the needed libraries                                                   #
#                                                                                   #
#####################################################################################

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser

## NASA Mars News

In [3]:
#####################################################################################
#                                                                                   #
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph    #
# Text. Assign the text to variables that you can reference later.                  #
#                                                                                   #
#####################################################################################

url = "https://mars.nasa.gov/news/"
response = requests.get(url)
soup = bs(response.text, 'html.parser')

results = soup.find_all('div', class_='slide')

news_title = []
news_p = []

for result in results:
    news_title.append(result.find_all('div', class_='content_title')[0].find('a').text)
    news_p.append(result.find_all('div', class_='rollover_description_inner')[0].text)

## JPL Mars Space Images - Featured Image

In [4]:
#####################################################################################
#                                                                                   #
# Visit the url for JPL Featured Space Image here.                                  #
#                                                                                   #
# Use splinter to navigate the site and find the image url for the current Featured #
# Mars Image and assign the url string to a variable called featured_image_url.     #
#                                                                                   #
# Make sure to find the image url to the full size .jpg image.                      #
#                                                                                   #
# Make sure to save a complete url string for this image.                           #
#                                                                                   #
#####################################################################################

executable_path = {'executable_path': '/drivers/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

browser.click_link_by_partial_text('FULL IMAGE')
browser.find_by_css('a[title="Display actual size"]').click()
featured_image_url = browser.find_by_css('img[class="fancybox-image"]')['src']
browser.quit()

## Mars Weather

In [5]:
#####################################################################################
#                                                                                   #
# Visit the Mars Weather twitter account here and scrape the latest Mars weather    #
# tweet from the page. Save the tweet text for the weather report as a variable     #
# called mars_weather.                                                              #
#                                                                                   #
#####################################################################################

url='https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

results = soup.find_all('div', class_='js-tweet-text-container')

mars_weather = []

for result in results:
    mars_weather.append(result.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')[0].text)

## Mars Facts

In [6]:
#####################################################################################
#                                                                                   #
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing   #
# facts about the planet including Diameter, Mass, etc.                             #
#                                                                                   #
# Use Pandas to convert the data to a HTML table string.                            #
#                                                                                   #
#####################################################################################

url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)

df = tables[0]
df.columns = ["Fact", "Value"]
df.set_index('Fact', inplace=True)
    
html_table = df.to_html()
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Fact</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Mars Hemispheres

In [5]:
#####################################################################################
#                                                                                   #
# Visit the USGS Astrogeology site here to obtain high resolution images for each   #
# of Mar's hemispheres.                                                             #
#                                                                                   #
# You will need to click each of the links to the hemispheres in order to find the  #
# image url to the full resolution image.                                           #
#                                                                                   #
# Save both the image url string for the full resolution hemisphere image, and the  #
# Hemisphere title containing the hemisphere name. Use a Python dictionary to store #
# the data using the keys img_url and title.                                        #
#                                                                                   #
# Append the dictionary with the image url string and the hemisphere title to a list#
# This list will contain one dictionary for each hemisphere.                        #
#                                                                                   #
#####################################################################################

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(url)
soup = bs(response.text, 'html.parser')

results = soup.find_all('div', class_='item')

hemis_title = []
hemis_url = []

for result in results:
    hemis_title.append(result.find_all('div', class_='description')[0].find('h3').text)
    
executable_path = {'executable_path': '/drivers/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url)

for hem in hemis_title:
    browser.click_link_by_partial_text(hem)
    response = requests.get(browser.url)
    soup = bs(response.text, 'html.parser')
    results = soup.find_all('li')
    for result in results:
        if result.find_all('a')[0].text == "Sample":
            hemis_url.append(result.find_all('a')[0]['href'])
    browser.back()
    
hemisphere_image_urls = []

for x in range (0, 4):
    myDict = {"title": hemis_title[x], "img_url": hemis_url[x]}
    hemisphere_image_urls.append(myDict)
browser.quit()